In [1]:
from pathlib import Path
import os
import shutil
import torch
import pandas as pd

from input_template import UserInputArgs
from save_lstm import load_lstm_info

import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

In [2]:
model = torch.load("recently_trained_model.pt")

In [4]:
args = UserInputArgs()
args.model_load_filename = "MED2_30"
network, std_factors = load_lstm_info(args)

Loading LSTM Saved in:  MED2_30 .pickle


In [7]:
## Setup the standard user inputs

print("Cuda available:", torch.cuda.is_available())
if torch.cuda.is_available():
    torch.cuda.empty_cache()  # to avoid some rare errors.

print("Cuda available:", torch.cuda.is_available())
if torch.cuda.is_available():
    torch.cuda.empty_cache()  # to avoid some rare errors.

# Check if using CPU or GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set parameters
args = UserInputArgs()
plot_args = PlottingArgs()
data_info_args = DataInfoArgs()
save_data_args = SaveDataArgs()
derived_args = DerivedArgs(args, data_info_args)

# Read in and Standardize the data. Each input & target is formatted:
# [num_realizations, full series length (17990), num_parameters]
print("\nLoading Training Data")
if args.training_mode == False:
    train_input, train_target, train_sc = load_and_standardize(
        data_info_args.train_sc, data_info_args.train_lamp, args, std_factors
    )
elif args.training_mode == True:
    train_input, train_target, std_factors, train_sc = load_and_standardize(
        data_info_args.train_sc, data_info_args.train_lamp, args
    )
print("\nLoading Validation Data")
val_input, val_target, val_sc = load_and_standardize(
    data_info_args.val_sc, data_info_args.val_lamp, args, std_factors
)
print("\nLoading Testing Data")
test_input, test_target, test_sc = load_and_standardize(
    data_info_args.test_sc, data_info_args.test_lamp, args, std_factors
)

print("standardization factors are", std_factors)

if args.training_mode == True:
    # create an instance of our LSTM network
    network = LSTM(
        args.input_size,
        args.hidden_size,
        args.num_layers,
        args.output_size,
        args.bi_directional,
        args.dropout,
    ).to(device)

# Reshape the data to take into account the time resolution
train_input, train_target = reshape_for_time_resolution(train_input, train_target, args)
val_input, val_target = reshape_for_time_resolution(val_input, val_target, args)
test_input, test_target = reshape_for_time_resolution(test_input, test_target, args)

# Create Dataset objects for each of our train/val/test sets
train_dataset = S2LDataset(train_input, train_target)
val_dataset = S2LDataset(val_input, val_target)
test_dataset = S2LDataset(test_input, test_target)

# Create a PyTorch dataloader for each train/val set. Test set isn't needed until later
train_loader = DataLoader(
    train_dataset, batch_size=derived_args.train_batch_size, shuffle=True
)
val_loader = DataLoader(val_dataset, batch_size=derived_args.val_batch_size)
test_loader = DataLoader(test_dataset, batch_size=derived_args.test_batch_size)

# Display dataset information
print("\nReshaped the following data:")
print(f"train_input has shape	{train_input.shape}")
print(f"train_target has shape	{train_target.shape}")
print(f"val_input has shape 	{val_input.shape}")
print(f"val_target has shape 	{val_target.shape}")
print(f"test_input has shape 	{test_input.shape}")
print(f"test_target has shape 	{test_target.shape}")
realization_length = train_input.shape[1] * args.time_res

# initialize our optimizer. We'll use Adam
optimizer = torch.optim.Adam(network.parameters(), lr=args.lr)

OrderedDict([('lstm.weight_ih_l0',
              tensor([[ 1.5993e-01, -4.6538e-03, -1.4014e-01, -1.2784e-01],
                      [-3.9607e-01, -4.2352e-02, -5.9251e-02,  1.5927e-02],
                      [-7.1830e-03, -1.8736e-01,  5.4685e-01,  2.0289e-01],
                      [ 1.9613e-02, -1.0748e-01,  1.2186e-01,  1.7099e-01],
                      [-3.2579e-01, -2.5405e-01, -3.0665e-01,  8.3565e-02],
                      [ 2.8159e-01,  1.1357e-02, -1.7837e-01,  5.1188e-02],
                      [-1.1738e-01,  8.5709e-02, -8.8913e-03,  2.2782e-03],
                      [-1.7181e-01,  8.8365e-02,  4.3813e-01,  2.0528e-01],
                      [-3.2392e-02, -2.1326e-01, -4.0411e-01, -1.1053e-01],
                      [ 3.1405e-01,  4.7577e-01,  7.7157e-02, -4.6023e-01],
                      [ 1.1457e-01, -9.2223e-02,  5.4462e-01, -1.3585e-01],
                      [ 7.9956e-02,  2.7378e-03,  4.4426e-02,  2.5123e-01],
                      [ 1.7797e-01, -4.2349e-02, -2.0

In [11]:
# Update Model save name
args.model_save_filename ='MED3_50'

# Begin Training
if args.training_mode == True:
    best_val_loss = float("inf")
    best_loss_counter = 0
    print("\n Beginning Training")
    for epoch in range(1, args.epochs + 1):
        train_loss = train(network, device, train_loader, optimizer, args.train_fun_hyp)
        val_loss = test(network, device, val_loader, args.val_fun_hyp)
        if val_loss < 0.95 * best_val_loss:
            best_val_loss = val_loss
            best_loss_counter = 0
            torch.save(network.state_dict(), "recently_trained_model.pt")
        else:
            best_loss_counter += 1
        if best_loss_counter > 30:
            break
        print(
            "Train Epoch: {:02d} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}".format(
                epoch, train_loss, val_loss
            )
        )
    print("Training Done\n")
    network.load_state_dict(
        torch.load("recently_trained_model.pt")
    )  # restoring the best found network based on validation data
    save_lstm_info(network.state_dict(), args, data_info_args, std_factors)

# Produce final LSTM output
train_loader = DataLoader(
    train_dataset, batch_size=derived_args.train_batch_size
)  # now with shuffle off so everything is ordered correctly
start_time = time.time()  # to show how long it takes to run the series through
train_lstm_output = test(
    network,
    device,
    train_loader,
    args.val_fun_hyp,
    derived_args.num_train_realizations,
    args.time_res,
    True,
)
val_lstm_output = test(
    network,
    device,
    val_loader,
    args.val_fun_hyp,
    derived_args.num_val_realizations,
    args.time_res,
    True,
)
test_lstm_output = test(
    network,
    device,
    test_loader,
    args.val_fun_hyp,
    derived_args.num_test_realizations,
    args.time_res,
    True,
)
end_time = time.time()
print("\ntrain output shape", train_lstm_output.shape)
print("val output shape  ", val_lstm_output.shape)
print("test output shape ", test_lstm_output.shape)
print(
    "Time to produce output for ",
    derived_args.num_realizations,
    " realizations:",
    (end_time - start_time),
)

# Reshape our input and targets to be same shape as output
train_input, train_target, train_lstm_output = reshape_full_series(
    train_input, train_target, train_lstm_output, args
)
val_input, val_target, val_lstm_output = reshape_full_series(
    val_input, val_target, val_lstm_output, args
)
test_input, test_target, test_lstm_output = reshape_full_series(
    test_input, test_target, test_lstm_output, args
)

# Unstandardize
train_target, train_lstm_output = unstandardize_all_data(
    train_target, train_lstm_output, std_factors, args
)
val_target, val_lstm_output = unstandardize_all_data(
    val_target, val_lstm_output, std_factors, args
)
test_target, test_lstm_output = unstandardize_all_data(
    test_target, test_lstm_output, std_factors, args
)

# Print Final Errors
print("\nSimpleCode Error Results:")
print_error_report(
    train_sc[:, :realization_length, :],
    val_sc[:, :realization_length, :],
    test_sc[:, :realization_length, :],
    train_target,
    val_target,
    test_target,
    args,
)
print("\nLSTM Error Results:")
print_error_report(
    train_lstm_output,
    val_lstm_output,
    test_lstm_output,
    train_target,
    val_target,
    test_target,
    args,
)

# Change save_data_args.test to a list of equal length to cases_file tests
save_data_args.test = list(range(0, len(data_info_args.test_sc)))
print(len(save_data_args.test))

# Save Results
save_lstm_results(
    train_lstm_output,
    val_lstm_output,
    test_lstm_output,
    save_data_args,
    data_info_args,
    args,
    std_factors,
)

lstm.weight_ih_l0 tensor([[-2.8284e-01, -1.4879e-01,  8.7982e-02,  4.9299e-01],
        [-8.1963e-02,  1.7688e-01, -1.0212e-01,  3.1205e-01],
        [-5.1556e-02,  1.3836e-01,  1.7581e-01,  2.0124e-01],
        [ 1.7402e-02, -2.0676e-01,  4.7735e-01,  1.4529e-01],
        [-1.2323e-01,  6.0202e-01,  1.9278e-01,  4.0765e-01],
        [ 3.5555e-02, -2.8023e-02, -4.4381e-01, -6.1289e-02],
        [ 1.6347e-01, -1.0520e-01,  7.5976e-01, -1.4263e-01],
        [ 4.7112e-01,  1.1196e-01,  6.3340e-02,  2.8327e-01],
        [ 1.5249e-02, -1.9262e-01,  1.4825e-02,  4.6228e-01],
        [ 1.0571e-01,  7.3356e-02,  4.8328e-01, -8.5663e-02],
        [ 1.8802e-01, -8.3461e-02, -2.7941e-01,  1.4609e-01],
        [-2.0915e-01, -1.3923e-01, -3.7490e-01, -2.5929e-01],
        [ 1.8923e-01,  2.3874e-01, -8.5566e-02, -2.5886e-01],
        [ 7.1491e-03,  2.0388e-01, -1.8319e-01,  1.1668e-01],
        [-2.8784e-01,  1.5771e-01,  1.4037e-01,  4.5798e-01],
        [ 7.8961e-02,  1.1134e-01, -1.9722e-01,  2.5